In [2]:
import zarr
import stackstac
import xarray as xr
import numpy as np
import geopandas as gpd
import planetary_computer as pc
from dask.distributed import Client
from shapely.geometry import mapping, box
from pystac_client import Client as pystac_client

#### Set up dask client

In [3]:
client = Client(
    threads_per_worker=1,
    memory_limit="4GB"
)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 20
Total threads: 20,Total memory: 74.51 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:56685,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:56763,Total threads: 1
Dashboard: http://127.0.0.1:56799/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:56688,


Task exception was never retrieved
future: <Task finished name='Task-810866' coro=<Client._gather.<locals>.wait() done, defined at c:\Users\geo1k\.local\share\mamba\envs\geodask\Lib\site-packages\distributed\client.py:2385> exception=AllExit()>
Traceback (most recent call last):
  File "c:\Users\geo1k\.local\share\mamba\envs\geodask\Lib\site-packages\distributed\client.py", line 2394, in wait
    raise AllExit()
distributed.client.AllExit


#### Prep Input data:
- Read data
- Partition into 0.5x0.5 degree tiles

In [4]:
fergana_gdf = gpd.read_file("../data/Train/Fergana_training_samples.shp")
fergana_gdf = fergana_gdf.set_crs(4326)

In [5]:
min_x, min_y, max_x, max_y = [round(coord) for coord in fergana_gdf.total_bounds.tolist()]
cell_size = 0.5
x_cells = int((max_x - min_x) / cell_size)
y_cells = int((max_y - min_y) / cell_size)

polygons = []
tile_ids = []
for i in range(x_cells):
    for j in range(y_cells):

        x1 = min_x + i * cell_size
        y1 = min_y + j * cell_size

        poly = box(x1, y1, x1 + cell_size, y1 + cell_size)
        tile_id = f"{x1}_{y1 + cell_size}"
        
        polygons.append(poly)
        tile_ids.append(tile_id)

tiles = gpd.GeoDataFrame({"tile_id": tile_ids, "geometry": polygons}, crs="EPSG:4326")

In [6]:
# Join plots to 0.5° grid cells
joined_gdf = fergana_gdf.sjoin(tiles)

#### Search Sentinel-2 scenes for the first partition

In [8]:
first_part_df = joined_gdf[joined_gdf["tile_id"] == "72.5_41.0"]
bounds = first_part_df.total_bounds.tolist()

In [9]:
# Sign into Microsoft planetary computer
stac_url = "https://planetarycomputer.microsoft.com/api/stac/v1"
sentinel2_client = pystac_client.open(
    stac_url,
    modifier=pc.sign_inplace
)

In [14]:
bands = ["B02", "B03", "B04", "B05", "B07", "B08", "B8A", "B11"]

# Query Sentinel-2 items for Fergana region (2020-2024)
search = sentinel2_client.search(
    collections=["sentinel-2-l2a"],
    bbox=bounds,
    datetime="2024-01-01/2024-12-31",
)

items = list(search.items())
print(f"Found {len(items)} Sentinel-2 items")

Found 420 Sentinel-2 items


#### Build the monthly data cube

In [15]:
datacube = stackstac.stack(
    items,
    bounds_latlon=bounds,
    assets=bands,
    epsg=4326,
    snap_bounds=False,
)
datacube

<xarray.DataArray 'stackstac-3efd6b1c6e705ca63f797d79fe49795d' (time: 420,
                                                                band: 8,
                                                                y: 11044,
                                                                x: 3995)> Size: 1TB
dask.array<fetch_raster_window, shape=(420, 8, 11044, 3995), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/43)
  * time                                     (time) datetime64[ns] 3kB 2024-0...
    id                                       (time) <U54 91kB 'S2A_MSIL2A_202...
  * band                                     (band) <U3 96B 'B02' ... 'B11'
  * x                                        (x) float64 32kB 72.51 ... 72.99
  * y                                        (y) float64 88kB 40.99 ... 40.52
    s2:product_uri                           (time) <U65 109kB 'S2A_MSIL2A_20...
    ...                                       ...
    title                                    (band) <U37 1kB 'Band 2 - Blue -...
    gsd                                      (band) float64 64B 10.0 ... 20.0
    common_name                              (band) <U7 224B 'blue' ... 'swir16'
    center_wavelength                        (band) float64 64B 0.49 ... 1.61
    full_width_half_max                      (band) float64 64B 0.098 ... 0.143
    epsg                                     int64 8B 4326
Attributes:
    spec:           RasterSpec(epsg=4326, bounds=[72.5111022770861, 40.518846...
    crs:            epsg:4326
    transform:      | 0.00, 0.00, 72.51|\n| 0.00,-0.00, 40.99|\n| 0.00, 0.00,...
    resolution_xy:  (0.00011885348816029049, 4.290962659380655e-05)

In [19]:
monthly_datacube = datacube.resample(time="1ME").median()
monthly_datacube = monthly_datacube.assign_coords({"year_month": monthly_datacube["time"].dt.strftime("%Y-%m")})
monthly_datacube

c:\Users\geo1k\.local\share\mamba\envs\geodask\Lib\site-packages\dask\array\core.py:1792: PerformanceWarning: Increasing number of chunks by factor of 10
  return da_func(*args, **kwargs)


<xarray.DataArray 'stackstac-3efd6b1c6e705ca63f797d79fe49795d' (time: 12,
                                                                band: 8,
                                                                y: 11044,
                                                                x: 3995)> Size: 34GB
dask.array<stack, shape=(12, 8, 11044, 3995), dtype=float64, chunksize=(1, 1, 632, 632), chunktype=numpy.ndarray>
Coordinates: (12/18)
  * band                                     (band) <U3 96B 'B02' ... 'B11'
  * x                                        (x) float64 32kB 72.51 ... 72.99
  * y                                        (y) float64 88kB 40.99 ... 40.52
    s2:datatake_type                         <U8 32B 'INS-NOBS'
    s2:saturated_defective_pixel_percentage  float64 8B 0.0
    s2:product_type                          <U7 28B 'S2MSI2A'
    ...                                       ...
    common_name                              (band) <U7 224B 'blue' ... 'swir16'
    center_wavelength                        (band) float64 64B 0.49 ... 1.61
    full_width_half_max                      (band) float64 64B 0.098 ... 0.143
    epsg                                     int64 8B 4326
  * time                                     (time) datetime64[ns] 96B 2024-0...
    year_month                               (time) object 96B '2024-01' ... ...
Attributes:
    spec:           RasterSpec(epsg=4326, bounds=[72.5111022770861, 40.518846...
    crs:            epsg:4326
    transform:      | 0.00, 0.00, 72.51|\n| 0.00,-0.00, 40.99|\n| 0.00, 0.00,...
    resolution_xy:  (0.00011885348816029049, 4.290962659380655e-05)

#### Add agricultural indices

In [20]:
def normalized_difference(b1: xr.DataArray, b2: xr.DataArray, name: str):
    ndi = (b1 - b2) / (b1 + b2)
    ndi.name = name
    return ndi

def calc_evi(nir: xr.DataArray, red: xr.DataArray, blue: xr.DataArray):
    evi = 2.5 * (
        (nir - red) /
        (nir + 6 * red - 7.5 * blue + 1)
    )
    evi.name = "evi"
    return evi.drop_vars("band")

def calc_chlorophyll(red_edge: xr.DataArray, narrow_nir: xr.DataArray, name: str):
    chlorophyll = (narrow_nir - red_edge) / red_edge
    chlorophyll.name = name
    return chlorophyll.drop_vars("band")


In [21]:
blue        = monthly_datacube.sel(band="B02") / 10000
green       = monthly_datacube.sel(band="B03") / 10000
red         = monthly_datacube.sel(band="B04") / 10000
red_edge_5  = monthly_datacube.sel(band="B05") / 10000
red_edge_7  = monthly_datacube.sel(band="B07") / 10000
nir         = monthly_datacube.sel(band="B08") / 10000
narrow_nir  = monthly_datacube.sel(band="B8A") / 10000
swir        = monthly_datacube.sel(band="B11") / 10000

monthly_datacube["ndvi"] = normalized_difference(nir, red, "ndvi")
monthly_datacube["evi"] = calc_evi(nir, red, blue)
monthly_datacube["ndwi"] = normalized_difference(green, nir, "ndwi")
monthly_datacube["ndmi"] = normalized_difference(nir, swir, "ndmi")
monthly_datacube["wet_chlorophyll"] = calc_chlorophyll(red_edge_5, narrow_nir, "wet_chlorophyll")
monthly_datacube["canopy_chlorophyll"] = calc_chlorophyll(red_edge_7, narrow_nir, "canopy_chlorophyll")
monthly_datacube["ndre"] = normalized_difference(nir, red_edge_5, "ndre")

In [25]:
monthly_datacube

<xarray.DataArray 'stackstac-3efd6b1c6e705ca63f797d79fe49795d' (time: 12,
                                                                band: 8,
                                                                y: 11044,
                                                                x: 3995)> Size: 34GB
dask.array<stack, shape=(12, 8, 11044, 3995), dtype=float64, chunksize=(1, 1, 632, 632), chunktype=numpy.ndarray>
Coordinates: (12/25)
  * band                                     (band) <U3 96B 'B02' ... 'B11'
  * x                                        (x) float64 32kB 72.51 ... 72.99
  * y                                        (y) float64 88kB 40.99 ... 40.52
    s2:datatake_type                         <U8 32B 'INS-NOBS'
    s2:saturated_defective_pixel_percentage  float64 8B 0.0
    s2:product_type                          <U7 28B 'S2MSI2A'
    ...                                       ...
    evi                                      (time, y, x) float64 4GB dask.array<chunksize=(1, 632, 632), meta=np.ndarray>
    ndwi                                     (time, y, x) float64 4GB dask.array<chunksize=(1, 632, 632), meta=np.ndarray>
    ndmi                                     (time, y, x) float64 4GB dask.array<chunksize=(1, 632, 632), meta=np.ndarray>
    wet_chlorophyll                          (time, y, x) float64 4GB dask.array<chunksize=(1, 632, 632), meta=np.ndarray>
    canopy_chlorophyll                       (time, y, x) float64 4GB dask.array<chunksize=(1, 632, 632), meta=np.ndarray>
    ndre                                     (time, y, x) float64 4GB dask.array<chunksize=(1, 632, 632), meta=np.ndarray>
Attributes:
    spec:           RasterSpec(epsg=4326, bounds=[72.5111022770861, 40.518846...
    crs:            epsg:4326
    transform:      | 0.00, 0.00, 72.51|\n| 0.00,-0.00, 40.99|\n| 0.00, 0.00,...
    resolution_xy:  (0.00011885348816029049, 4.290962659380655e-05)

In [24]:
coords = [(geom.x, geom.y) for geom in first_part_df.geometry]
lons, lats = zip(*coords)

# Vectorized selection - much more efficient than looping
point_data = datacube.sel(
    x=list(lons),
    y=list(lats),
    method="nearest"
)

point_data = point_data.assign_coords(point_id=("x", first_part_df["ID"].values))
point_data

<xarray.DataArray 'stackstac-3efd6b1c6e705ca63f797d79fe49795d' (time: 420,
                                                                band: 8, y: 32,
                                                                x: 32)> Size: 28MB
dask.array<getitem, shape=(420, 8, 32, 32), dtype=float64, chunksize=(1, 1, 32, 32), chunktype=numpy.ndarray>
Coordinates: (12/44)
  * time                                     (time) datetime64[ns] 3kB 2024-0...
    id                                       (time) <U54 91kB 'S2A_MSIL2A_202...
  * band                                     (band) <U3 96B 'B02' ... 'B11'
  * x                                        (x) float64 256B 72.94 ... 72.97
  * y                                        (y) float64 256B 40.84 ... 40.89
    s2:product_uri                           (time) <U65 109kB 'S2A_MSIL2A_20...
    ...                                       ...
    gsd                                      (band) float64 64B 10.0 ... 20.0
    common_name                              (band) <U7 224B 'blue' ... 'swir16'
    center_wavelength                        (band) float64 64B 0.49 ... 1.61
    full_width_half_max                      (band) float64 64B 0.098 ... 0.143
    epsg                                     int64 8B 4326
    point_id                                 (x) int32 128B 12 26 56 ... 486 500
Attributes:
    spec:           RasterSpec(epsg=4326, bounds=[72.5111022770861, 40.518846...
    crs:            epsg:4326
    transform:      | 0.00, 0.00, 72.51|\n| 0.00,-0.00, 40.99|\n| 0.00, 0.00,...
    resolution_xy:  (0.00011885348816029049, 4.290962659380655e-05)

In [26]:
desired_coords = ["time", "x", "y", "band", "id", "year_month", "ndvi", "evi", "ndwi", "ndmi", "wet_chlorophyll", "canopy_chlorophyll", "ndre"]
cleaned_point_data = point_data.drop_vars(
        [coord for coord in point_data.coords if coord not in desired_coords],
        errors="ignore"
)
cleaned_point_data = cleaned_point_data.drop_attrs()
cleaned_point_data

<xarray.DataArray 'stackstac-3efd6b1c6e705ca63f797d79fe49795d' (time: 420,
                                                                band: 8, y: 32,
                                                                x: 32)> Size: 28MB
dask.array<getitem, shape=(420, 8, 32, 32), dtype=float64, chunksize=(1, 1, 32, 32), chunktype=numpy.ndarray>
Coordinates:
    id       (time) <U54 91kB 'S2A_MSIL2A_20240103T060241_R091_T43TCF_2024010...
  * time     (time) datetime64[ns] 3kB 2024-01-03T06:02:41.024000 ... 2024-12...
  * band     (band) <U3 96B 'B02' 'B03' 'B04' 'B05' 'B07' 'B08' 'B8A' 'B11'
  * x        (x) float64 256B 72.94 72.57 72.84 72.6 ... 72.99 72.94 72.8 72.97
  * y        (y) float64 256B 40.84 40.94 40.81 40.87 ... 40.99 40.99 40.89

In [27]:
cleaned_point_data.to_zarr("part1_2.zarr", mode="w", zarr_format=2)

c:\Users\geo1k\.local\share\mamba\envs\geodask\Lib\site-packages\distributed\client.py:3371: UserWarning: Sending large graph of size 26.13 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
import pandas as pd

# Load the zarr data
data = xr.open_zarr("part1_2.zarr")

# Reshape to create the wide format
def create_wide_dataframe(data):
    """Convert xarray to wide dataframe with columns for each index-month combination"""

    # Convert to pandas with multi-index columns
    df = data.to_dataframe().unstack(['time', 'band'])

    # Flatten column names to format like 'NDVI_2024-01', 'EVI_2024-01', etc.
    df.columns = [f"{band}_{time.strftime('%Y-%m')}"
                for time, band in df.columns]

    # Reset index to get point_id as a column
    df = df.reset_index()

    # Add the original point metadata (cropland labels, coordinates, etc.)
    df = df.merge(first_part_df[['ID', 'Cropland']],
                left_on='point_id', right_on='ID', how='left')

    return df

# Create the wide dataframe
wide_df = create_wide_dataframe(data)
print(f"DataFrame shape: {wide_df.shape}")
print("Columns:", wide_df.columns.tolist()[:10])  # Show first 10 columns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Prepare features and target
X = wide_df.drop(['point_id', 'ID', 'Cropland', 'x', 'y'], axis=1)
y = wide_df['Cropland']

# Handle any NaN values
X = X.fillna(X.median())

# Scale features (important for LogReg)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Fit LogisticRegression
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(X_train, y_train)

# Evaluate
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))

In [28]:
client.close()

2025-09-19 20:51:50,668 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
